Now that we have a dataset, we can start to do some actual analysis. I'm going to be attempting to replicate the methodology of this paper:

Sapienza, Anna and Goyal, Palash and Ferrara, Emilio. Deep Neural Networks for Optimal Team Composition. Frontiers in Big Data, vol 2. Jun 2019. https://arxiv.org/abs/1805.03285 

While roller derby and esports games like League of Legends obviously are very different, in many ways, they can be treated similarly- each League match and individual jam of a derby bout consists of a team of 5 players with different defined roles attempting to achieve an objective while slowing the opposing team's attempt to achieve theirs.

A derby bout (game) consists of a series of many individual jams. Each team forwards a defensive line of four "blockers" and an offensive line of one "jammer". The jammer scores points by passing through the "pack" of blockers- one initial non-scoring pass through the pack is required, and then one point is earned for each of the opposing team's blockers that the jammer passes on subsequent laps. Each jam can run for a set amount of time, but the jammer that is the first to complete the non-scoring pass ("lead jammer") can choose to end the jam early. In addition, the jammer can hand off their jammer status to one special blocker on each team called a "pivot" by passing the special helmet cover that the jammer wears. This is the general gist of the sport- in many ways, it's similar to the playground game "Red Rover", but on wheels.

Naturally, when the blockers try to stop the jammer, things can get scrappy! Various penalties are given when a player shoves another in an illegal manner, when a blocker strays too far from the pack, when a player goes out of bounds, when a blocker makes an illegal formation (such as linking arms with another blocker), etc. It's general "derby wisdom" that certain penalties are more common "new-skater" penalties, while the distribution of penalties changes with skill. We can test this!


Let's pick a team. I'll use the Kalamazoo Derby Darlins, the team I've announced for for the past few years. 

In this analysis, I'm going to make some assumptions.
-First, that the fundamental unit of derby is not the bout, but the jam. Each jam is unique, and may have starting conditions determined by the preceding jam, but ultimately, for the purposes of this analysis, the only influence jam 1 may have on a jam like jam 20 is player stamina (N.B.: sometimes players can still be in the penalty box from previous jams, so this is not strictly correct! but it's probably correct enough for what we'd like to test here). This means that I will update a player's "rating" each jam rather than each bout.

-Second, that the "figure of merit" to determine the performance of a jammer is the total number of points they score in a jam, but that the "figure of merit" to determine the performance of a blocker line is the difference between their jammer's score and the opposing jammer's score. A good blocker line is able to slow the opposing jammer substantially while also letting their own through.

-Third: the rules of roller derby change often, as the sport is still relatively new. For instance- at one point, jammers scored an additional point for passing the opposing team's jammer as well as blockers.
    

In [1]:
import requests
import pandas as pd
import numpy as np
import trueskill
from bs4 import BeautifulSoup
from itertools import product
from urllib.request import urlopen
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph 
import matplotlib.pyplot as plt
import pylab

import nbimporter
import Webscraper as wsc


teamID=str(3637)
teamName='Killamazoo'

In [2]:
#First, get the lineups for each jam KDD has stats available for.
AllLineups = wsc.GetAllLineups(teamID, teamName)

# Also, get expanding average of score differentials for each jam. We'll use a player's
# average score differential after a given jam as a proxy for their skill ranking as measured
# after playing that jam.

AllDiffs = wsc.AvgChangeFromJam(teamID, teamName)

Webscraper.ipynb:7: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 7 of the file Webscraper.ipynb. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  "In this series of notebooks, I will attempt to do some introductory exploration of various roller derby statistics. We will use the publicly available stats on the FlatTrackStats website. First, I will build a table scraper tool using the BeautifulSoup4 package to parse the stats tables on the website. If not already installed, you will need pandas and BeautifulSoup4 in order to run this notebook. "
Webscraper.ipynb:16: GuessedAtParserWarning: No parser was explicitly specified, so I'm u

Let's only look at blockers for now, since they interact most closely with each other. Matching jammers to blocker lines is a different question than composing the lines themselves, since interplay is different.

In [3]:
print(AllLineups, AllDiffs)

              Jammer     Jstats              B1              B2  \
0         Beaver Jam      Lead   Aly-Kate Co...  Wreck Keene...   
1       Buns N Roses          0  Painbow Con...  Smash Bandi...   
2        Weers Waldo          0  Painbow Con...  Aly-Kate Co...   
3      Hill-De-Beast          0  Smash Bandi...   Beverly Hells   
4         Beaver Jam          0  Smash Bandi...  Wreck Keene...   
...              ...        ...             ...             ...   
1063   Beverly Hells          0  Sparkills (...  Rosie Feroc...   
1064  Delilah Danger      Lead          Javelin  Ivanna O'Bl...   
1065  Rosie Feroc...          0       Lady Hawk   Noam Stompsky   
1066  Sparkills (...  LeadLoss   Ivanna O'Bl...  Ramona D. F...   
1067   Beverly Hells          0         Javelin  Delilah Danger   

                  B3              B4  jamscore  runscore  ScoreDiff  
0            Mustang         Javelin        10        10       10.0  
1     Maggie Walters  Ophelia Plenty         0        1

Next, let's convert each jam into a weighted directed graph. 

In [12]:
blockerlines = AllLineups[['B1', 'B2', 'B3', 'B4']]
#print(blockerlines)

for jamnum in range(len((blockerlines.index))):
    G = nx.complete_graph(4, nx.DiGraph())
    blockers = blockerlines.iloc[jamnum].to_list()
    mapping = dict(zip(G, blockers))
    G = nx.relabel_nodes(G, mapping)
    
    for edge in G.edges():
        weight = AllDiffs.iloc[jamnum][edge[0]]
        #print(weight)
        G[edge[0]][edge[1]]['weight'] = weight
        nx.write_weighted_edgelist(G, "Data/"+teamID+str(jamnum)+".edgelist")